https://www.cnblogs.com/wwj99/p/12503545.html
BPE 算法

GPT-2 模型在数据预处理时使用了字节对编码（Byte Pair Encoding，简称 BPE）方法，BPE 是一种能够解决未登录词问题，并减小词典大小的方法。它综合利用了单词层面编码和字符层面编码的优势，举例来说，我们要对下面的字符串编码，

aaabdaaabac
字节对 aa 出现的次数最多，所以我们将它替换成一个没在字符串中被用过的字符 Z ，

ZabdZabac
Z=aa
然后我们重复这个过程，用 Y 替换 ab ，

ZYdZYac
Y=ab
Z=aa
继续，用 X 替换 ZY ，

XdXac
X=ZY
Y=ab
Z=aa

In [1]:
import re
import collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq  # 计算字节对出现频率
    return pairs


def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))  # 将字节对中可解释为正则运算符的字符转义
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')  # 将要合并的字节对前后只能为空白字符
    for word in v_in:
        w_out = p.sub(''.join(pair), word)  # 合并符合条件的字节对
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>': 5, 'l o w e r </w>': 2,
         'n e w e s t </w>': 6, 'w i d e s t </w>': 3}
num_merges = 10
for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 选择频率最大的字节对
    vocab = merge_vocab(best, vocab)
    print(best)
print(vocab)


('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [2]:
# top-k
import random

def select_top_k(predictions, k=10):
    predicted_index = random.choice(
        predictions[0, -1, :].sort(descending=True)[1][:10]).item()
    return predicted_index

In [3]:
# 使用在 PyTorch-Transformers 模型库中封装好的 GPT2Tokenizer() 和 GPT2LMHeadModel()
# 安装 PyTorch-Transformers
# !pip install pytorch_transformers==1.0
# !pip install pytorch_transformers==1.0 -i  https://pypi.tuna.tsinghua.edu.cn/simple/

In [5]:
import torch
from pytorch_transformers import BertModel, GPT2Model, GPT2Tokenizer

import logging
logging.basicConfig(level=logging.INFO)

# 载入预训练模型的分词器
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 使用 GPT2Tokenizer 对输入进行编码
text = "Yesterday, a man named Jack said he saw an alien,"
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor.shape

INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt not found in cache, downloading to C:\Users\fuliu\AppData\Local\Temp\tmp5sa6c9a8

100%|██████████| 456318/456318 [15:27<00:00, 491.80B/s][A
INFO:pytorch_transformers.file_utils:copying C:\Users\fuliu\AppData\Local\Temp\tmp5sa6c9a8 to cache at C:\Users\fuliu\.cache\torch\pytorch_transformers\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:pytorch_transformers.file_utils:creating metadata file for C:\Users\fuliu\.cache\torch\pytorch_transformers\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:pytorch_transformers.file_utils:removing temp file C:\Users\fuliu\AppData\Local\Temp\tmp5sa6c9a8
INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json fr

torch.Size([1, 12])

In [ ]:
print(indexed_tokens)


In [ ]:
from pytorch_transformers import GPT2LMHeadModel

# 读取 GPT-2 预训练模型
# model = GPT2LMHeadModel.from_pretrained("./")
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

total_predicted_text = text
n = 100  # 预测过程的循环次数
for _ in range(n):
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    predicted_index = select_top_k(predictions, k=10)
    predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
    total_predicted_text += tokenizer.decode(predicted_index)

    if '<|endoftext|>' in total_predicted_text:
        # 如果出现文本结束标志，就结束文本生成
        break

    indexed_tokens += [predicted_index]
    tokens_tensor = torch.tensor([indexed_tokens])

print(total_predicted_text)

微调生成戏剧文本
接下来，我们将使用一些戏剧剧本对 GPT-2 进行微调。由于 OpenAI 团队开源的 GPT-2 模型预训练参数为使用英文数据集预训练后得到的，虽然可以在微调时使用中文数据集，但需要大量数据和时间才会有好的效果，所以这里我们使用了英文数据集进行微调，从而更好地展现 GPT-2 模型的能力。

首先，下载训练数据集，这里使用了莎士比亚的戏剧作品《罗密欧与朱丽叶》作为训练样本。数据集已经提前下载好并放在云盘中，链接：https://pan.baidu.com/s/1LiTgiake1KC8qptjRncJ5w 提取码：km06

In [ ]:
with open('./romeo_and_juliet.txt', 'r') as f:
    dataset = f.read()

len(dataset)

In [ ]:
# 预处理训练集，将训练集编码、分段。
indexed_text = tokenizer.encode(dataset)
del(dataset)

dataset_cut = []
for i in range(len(indexed_text)//512):
    # 将字符串分段成长度为 512
    dataset_cut.append(indexed_text[i*512:i*512+512])
del(indexed_text)

dataset_tensor = torch.tensor(dataset_cut)
dataset_tensor.shape




# 这里使用 PyTorch 提供的 DataLoader() 构建训练集数据集表示，使用 TensorDataset() 构建训练集数据迭代器。
from torch.utils.data import DataLoader, TensorDataset

# 构建数据集和数据迭代器，设定 batch_size 大小为 2
train_set = TensorDataset(dataset_tensor,
                          dataset_tensor)  # 标签与样本数据相同
train_loader = DataLoader(dataset=train_set,
                          batch_size=2,
                          shuffle=False)
train_loader


# 检查是否机器有 GPU，如果有就在 GPU 运行，否则就在 CPU 运行。
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info('device: ', device)

# 开始训练
from torch import nn
from torch.autograd import Variable
import time

pre = time.time()

epoch = 30  # 循环学习 30 次

model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # 定义优化器

for i in range(epoch):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)

        optimizer.zero_grad()

        loss, logits, _ = model(data, labels=target)

        total_loss += loss

        loss.backward()
        optimizer.step()

        if batch_idx == len(train_loader)-1:
            # 在每个 Epoch 的最后输出一下结果
            print('average loss:', total_loss/len(train_loader))

print('训练时间：', time.time()-pre)

In [ ]:
# 训练结束后，可以使模型生成文本，观察输出。
text = "From fairest creatures we desire"  # 这里也可以输入不同的英文文本
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])

model.eval()
total_predicted_text = text

# 使训练后的模型进行 500 次预测
for _ in range(500):
    tokens_tensor = tokens_tensor.to('cuda')

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    predicted_index = select_top_k(predictions, k=10)

    predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
    total_predicted_text += tokenizer.decode(predicted_index)
    if '<|endoftext|>' in total_predicted_text:
        # 如果出现文本结束标志，就结束文本生成
        break

    indexed_tokens += [predicted_index]

    if len(indexed_tokens) > 1023:
        # 模型最长输入长度为1024，如果长度过长则截断
        indexed_tokens = indexed_tokens[-1023:]

    tokens_tensor = torch.tensor([indexed_tokens])

print(total_predicted_text)


